In [148]:
import numpy as np
import math
sqrt2 = math.sqrt(2.0)
davis_modified = True

def fine_tune(b0,b1,b2,b3):
    C1 = ( 1.5 * b2 - b1 ) >= (b1 * b2 * b3) * int(not davis_modified)
    C2 = b3 * ( 1.0 + b2 ) < 1/sqrt2
    
    C3_bound = 1.25 * ( 1.0 - (1.8 + sqrt2) * b1 * b3 )
    C3 = b0 <= C3_bound
    
    k   = ( 1.0 + b0 - b1 ) / ( b0 + b1 )
    k_tilde = k / math.sqrt( 1.0 + math.pow(b3,2.0) * (k - 1.0) )
    C4 = k_tilde >= 1.0 + b2
    
    return (C1 and C2 and C3 and C4), C3_bound , math.pow(b1*b3/1.5,-3)

In [150]:
b0 = 0.1
grid = 200

b1 = np.linspace(0,0.5,num=grid+1,endpoint=False)[1:]
b2 = np.linspace(0,1/sqrt2,num=grid+1,endpoint=False)[1:]
b3 = np.linspace(0,1/sqrt2,num=grid+1,endpoint=False)[1:]

b1_mat = np.repeat(b1[:,np.newaxis], grid, axis=1)
b1_mat = np.repeat(b1_mat[:,:,np.newaxis], grid, axis=2)

b2_mat = np.repeat(b2[np.newaxis,:], grid, axis=0)
b2_mat = np.repeat(b2_mat[:,:,np.newaxis], grid, axis=2)

b3_mat = np.repeat(b3[np.newaxis,:], grid, axis=0)
b3_mat = np.repeat(b3_mat[np.newaxis,:,:], grid, axis=0)

kappa_mat   = ( 1.0 + b0 - b1_mat ) / ( b0 + b1_mat )
kappa_tilde = kappa_mat / np.sqrt( 1.0 + np.square(b3_mat) * (kappa_mat - 1.0) )

print(np.all(b1_mat[:,0,0] == b1))
print(np.all(kappa_tilde>0))

True
True


In [142]:
C1 = ( 1.5 * b2_mat - b1_mat ) >= b1_mat * b2_mat * b3_mat * int(not davis_modified)

C2 = b3_mat * ( 1.0 + b2_mat ) < 1/sqrt2

C3 = b0 <= 1.25 * ( 1.0 - (1.8 + sqrt2) * b1_mat * b3_mat )

C4 = kappa_tilde >= 1.0 + b2_mat

Call = np.logical_and.reduce((C1,C2,C3,C4))

M = np.max(b1_mat * b3_mat * Call.astype(int))
argM = np.unravel_index(np.argmax(b1_mat * b3_mat * Call.astype(int)),(grid,grid,grid))

In [143]:
print('beta_0: {:.4f}'.format(b0))
print('Good Param Ratio: {:.4f}'.format(np.sum(Call>0.5)/pow(grid,3)))
print()
print('Result coef: {:.4f}'.format(pow(M/1.5,-3)))
print('Result idx: {}'.format(argM))
print('Result param = (beta_1,beta_2,beta_3): ({:.4f}, {:.4f}, {:.4f})'.format(b1[argM[0]],b2[argM[1]],b3[argM[2]]))

beta_0: 0.1000
Good Param Ratio: 0.4514

Result coef: 289.6905
Result idx: (164, 77, 156)
Result param = (beta_1,beta_2,beta_3): (0.4104, 0.2744, 0.5523)


In [133]:
print(fine_tune(b0,0.4,0.32,0.53))

(True, 0.39823340597112966, 354.2143850292523)


In [163]:
gamma = 1.0e-6
delta = 0.02
sigma = 0.005

c = 32 * (math.pow(delta,4)+math.pow(sigma,4))
T = 500000
p = 100

b1    = 0.4

B = math.pow(c * p * math.log(T) / math.pow(gamma,2), 1/3)
print(B*gamma)
print(1.5 * math.pow(c*gamma*p*math.log(T),1/3)/ (b1 * delta))

0.0018894049969536276
0.35426343692880546


In [1]:
import math, random, argparse, time, sys
import numpy as np

from numba import jit

@jit(nopython=True)
def get_norm(U):
    dist = np.linalg.norm(U, ord=2)
    
    return dist

In [2]:
U = np.identity(5)

print(get_norm(U))

1.0
